In [1]:
from glob import glob
import json
import os
import re

def rebel_format(triplets):
    """
    Convert
    [['Bruno Santana', 'participant of', '2004 Summer Olympics'],
    ['Bruno Santana', 'participant of', '2008 Summer Olympics'],
    ['Bruno Santana', 'country of citizenship', 'Brazil']]
    to rebel format,
    <triplet> Bruno Santana <subj> 2004 Summer Olympics <obj> participant of <subj> 2008 Summer Olympics <obj> participant of <subj> Brazil <obj> country of citizenship
    """
    q = []
    for no, triple in enumerate(triplets):
        obj = ['<obj>'] + triple[1].split()
        subj = ['<subj>'] + triple[2].split()
        if no > 0 and triple[0] == triplets[no - 1][0]:
            q.extend(subj + obj)
        else:
            triplet = ['<triplet>'] + triple[0].split()
            q.extend(triplet + subj + obj)
    
    return re.sub(r'[ ]+', ' ', ' '.join(q)).strip()

In [2]:
files = ['kg-astroawani.jsonl', 'kg-paragraph-wikipedia.jsonl']

In [3]:
os.path.join('/home/ssd3/kg', files[0])

'/home/ssd3/kg/kg-astroawani.jsonl'

In [4]:
texts = []

with open(os.path.join('/home/husein/ssd3/kg', 'kg-astroawani.jsonl')) as fopen:
    for l in fopen:
        l = json.loads(l)
        
        if l['title_kg']:
            triples = []
            for t in l['title_kg']['triplets']:
                triples.append([t['subject'], t['predicate'], t['object']])
            kg = rebel_format(triples).strip()
            texts.append(kg)
                
        if l['description_kg']:
            triples = []
            for t in l['description_kg']['triplets']:
                triples.append([t['subject'], t['predicate'], t['object']])
            kg = rebel_format(triples).strip()
            texts.append(kg)
        
        for row in l['body_kg']:
            if row[1]:
                triples = []
                for t in row[1]['triplets']:
                    triples.append([t['subject'], t['predicate'], t['object']])
                kg = rebel_format(triples).strip()
                texts.append(kg)
len(texts)         

140163

In [5]:
with open(os.path.join('/home/husein/ssd3/kg', 'kg-paragraph-wikipedia.jsonl')) as fopen:
    for l in fopen:
        l = json.loads(l)
        for row in l['paragraph_kg']:
            if row[1]:
                triples = []
                for t in row[1]['triplets']:
                    triples.append([t['subject'], t['predicate'], t['object']])
                kg = rebel_format(triples).strip()
                texts.append(kg)
                
len(texts)

216895

In [6]:
texts = [t for t in texts if len(t)]
texts = list(set(texts))
len(texts)

213313

In [7]:
texts[0]

'<triplet> Perdagangan utama SPLASH <subj> air rawat <obj> penghasilan <subj> pukal <obj> dijual dalam bentuk <subj> Syarikat Bekalan Air Selangor (SYABAS) <obj> kepada <triplet> Syarikat Bekalan Air Selangor (SYABAS) <subj> Selangor dan Wilayah Persekutuan di Kuala Lumpur & Putrajaya <obj> menyelia pelanggan di'

In [8]:
with open('chatgpt-kg-triplets.jsonl', 'w') as fopen:
    for t in texts:
        fopen.write(f'{json.dumps(t)}\n')